In [1]:
# Importamos librerías.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
# sklearn.
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.preprocessing import label_binarize
# Hypertuning utilizando grid search.
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# Oversampler.
from imblearn.over_sampling import  RandomOverSampler
datos_url_pc= "C://Users/maxib/Source/repos/proyecto_final_coder/datos/Base_predictive_maintenance.csv"

In [2]:
# Leer .csv y creo un DF.
df = pd.read_csv(datos_url_pc, delimiter=",", index_col= ["UDI", "Product_ID"])
df.head()

,,Type,Air_temperature_[K],Process_temperature_[K],Rotational_speed_[rpm],Torque_[Nm],Tool_wear_[min],Target,Failure_Type
UDI,Product_ID,,,,,,,,
1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


# Definimos Variables 

In [3]:
# Definimos datos base.
data_x= df.drop(["Failure_Type", "Target"], axis=1)
data_y_ft= df["Failure_Type"]
data_y_t= df["Target"]
# Datos para testear.
data_test_x= pd.get_dummies(data_x)
data_test_y_ft_dum= pd.get_dummies(data_y_ft)

## Resampleo variable Failure type

In [4]:
# Oversampleamos variable Failure Type.
# Preparamos variable Target para entrenar los algoritmos.
X= data_x
y= data_y_ft
# Instantiating the random over sampler.
ros = RandomOverSampler(sampling_strategy={"Heat Dissipation Failure":1000,"Random Failures":1000,"Tool Wear Failure":1000,
                                            "Overstrain Failure":1000, "Power Failure":1000})
# Resampling X, y
X_ros_failure, y_ros_failure = ros.fit_resample(X, y)
X_ros_failure.info()
X_ros_failure["Type"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14652 entries, 0 to 14651
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Type                     14652 non-null  object 
 1   Air_temperature_[K]      14652 non-null  float64
 2   Process_temperature_[K]  14652 non-null  float64
 3   Rotational_speed_[rpm]   14652 non-null  int64  
 4   Torque_[Nm]              14652 non-null  float64
 5   Tool_wear_[min]          14652 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 686.9+ KB


array(['M', 'L', 'H'], dtype=object)

In [5]:
# Adquerimos Dummies de Failure Type (Para Train)
data_y_dum_train= pd.get_dummies(y_ros_failure)
data_X= pd.get_dummies(X_ros_failure)
data_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14652 entries, 0 to 14651
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Air_temperature_[K]      14652 non-null  float64
 1   Process_temperature_[K]  14652 non-null  float64
 2   Rotational_speed_[rpm]   14652 non-null  int64  
 3   Torque_[Nm]              14652 non-null  float64
 4   Tool_wear_[min]          14652 non-null  int64  
 5   Type_H                   14652 non-null  uint8  
 6   Type_L                   14652 non-null  uint8  
 7   Type_M                   14652 non-null  uint8  
dtypes: float64(3), int64(2), uint8(3)
memory usage: 615.4 KB


# Randon Forest Hiper Failure Type

In [6]:
rf= RandomForestClassifier() # Instanciamos el modelo
# UNO VS EL RESTO
model_rf= OneVsRestClassifier(rf)

# Definimos los parámetros de búsqueda
params = {
    'n_estimators' : [100, 200, 500],
    'max_features': [4,5,6,10]
}
grid_random_forest = GridSearchCV(estimator = rf,
                                  param_grid = params,
                                  scoring = 'neg_mean_absolute_error',
                                  cv = 5, 
                                  verbose = 1, # Muestra el resultado en pantalla
                                  n_jobs = -1) # corrida en paralelo

In [7]:
grid_random_forest.fit(data_X, data_y_dum_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\maxib\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\maxib\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\maxib\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\maxib\anaconda3\lib\site-packages\joblib\parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\maxib\anaconda3\lib\site-packages\jo

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_features': [4, 5, 6, 10],
                         'n_estimators': [100, 200, 500]},
             scoring='neg_mean_absolute_error', verbose=1)

In [8]:
#Obtenemos el mejor modelo!
grid_random_forest.best_estimator_
print("Mejores parametros: "+str(grid_random_forest.best_params_))
print("Mejor Score: "+str(grid_random_forest.best_score_)+'\n')
scores = pd.DataFrame(grid_random_forest.cv_results_)
scores

Mejores parametros: {'max_features': 5, 'n_estimators': 200}
Mejor Score: -5.052620374801706



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.853441,0.242901,0.185814,0.011089,4,100,"{'max_features': 4, 'n_estimators': 100}",-1.512226,-21.017741,-4.830774,-0.043686,-0.0,-5.480885,7.964547,4
1,5.920788,0.504214,0.357985,0.015352,4,200,"{'max_features': 4, 'n_estimators': 200}",-1.744513,-22.356022,-2.590330,-0.043686,-0.0,-5.346910,8.562704,3
2,15.336339,1.421107,0.968605,0.100620,4,500,"{'max_features': 4, 'n_estimators': 500}",-1.657625,-23.077846,-2.822241,-0.043686,-0.0,-5.520280,8.842332,6
3,3.577435,0.444901,0.195448,0.024423,5,100,"{'max_features': 5, 'n_estimators': 100}",-3.128511,-22.746389,-3.113993,-0.043686,-0.0,-5.806516,8.582616,8
4,6.953607,0.372661,0.419588,0.117043,5,200,"{'max_features': 5, 'n_estimators': 200}",-3.244911,-18.699932,-3.274573,-0.043686,-0.0,-5.052620,6.975626,1
5,17.087334,1.346982,0.861653,0.038310,5,500,"{'max_features': 5, 'n_estimators': 500}",-3.244683,-22.399124,-3.260182,-0.043686,-0.0,-5.789535,8.429542,7
6,4.115818,0.262772,0.177587,0.005116,6,100,"{'max_features': 6, 'n_estimators': 100}",-3.332367,-22.082452,-3.668089,-0.058248,-0.0,-5.828231,8.274735,9
7,7.562822,0.600785,0.343939,0.018689,6,200,"{'max_features': 6, 'n_estimators': 200}",-3.303764,-20.304845,-3.755631,-0.058191,-0.0,-5.484486,7.575114,5
8,16.180447,0.966895,0.624322,0.080118,6,500,"{'max_features': 6, 'n_estimators': 500}",-3.303651,-18.883032,-3.697554,-0.043686,-0.0,-5.185585,7.024320,2
9,0.079464,0.008452,0.000000,0.000000,10,100,"{'max_features': 10, 'n_estimators': 100}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10


In [10]:
#Predicción de casos nuevos
# Prediccion en Train
y_train_pred= grid_random_forest.predict(data_X)
# Prediccion en Test
y_test_pred= grid_random_forest.predict(data_test_x)
# 
y_score= grid_random_forest.best_estimator_.predict_proba(data_test_x)
#Accuracy
# print('Exactitud:', accuracy_score(data_y_ft, y_test_pred))

In [ ]:

# Binarize the output
roturas=['No Failure', 'Power Failure', 'Tool Wear Failure','Overstrain Failure', 'Random Failures','Heat Dissipation Failure']
y = label_binarize(data_y_ft, classes=roturas)
n_classes = y.shape[1]
y_ros= label_binarize(y_ros_failure, classes=roturas)

# Prediccion en Train
train_accuracy= accuracy_score(data_y_dum_train, y_train_pred)
print('% de aciertos sobre el set de evaluación:',train_accuracy)

# Prediccion en Test
test_accuracy = accuracy_score(data_test_y_ft_dum, y_test_pred)
print('% de aciertos sobre el set de evaluación:',test_accuracy)

# Matrix train
#plot_confusion_matrix(grid_random_forest, data_X, data_y_dum_train)
# Matrix test
#plot_confusion_matrix(grid_random_forest, data_test_x, data_test_y_ft_dum)
# plt.show()
# Recall con datos binarios
# For each class

precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y[:, i], y_score[:,i])
    average_precision[i] = average_precision_score(y[:, i], y_score[:, i])
# setup plot details
colors = itertools.cycle(["navy", "red", "darkorange", "cornflowerblue", "teal"])

_, ax = plt.subplots(figsize=(7, 8))

f_scores = np.linspace(0.2, 0.8, num=4)
lines, labels = [], []
for f_score in f_scores:
     x = np.linspace(0.01, 1)
     h = f_score * x / (2 * x - f_score)
     (l) = plt.plot(x[h >= 0], h[h >= 0], color="gray", alpha=0.2)
     plt.annotate("f1={0:0.1f}".format(f_score), xy=(0.9, h[45] + 0.02))


for i, color in zip(range(n_classes), colors):
    display = PrecisionRecallDisplay(
        recall=recall[i],
        precision=precision[i],
        average_precision=average_precision[i],
    )
    display.plot(ax=ax, name=f"Precision-recall for class {roturas[i]}", color=color)

# add the legend for the iso-f1 curves
handles, labels = display.ax_.get_legend_handles_labels()
handles.extend([l])
labels.extend(["iso-f1 curves"])
# set the legend and the axes
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.legend(handles=handles, labels=labels, loc="best")
ax.set_title("Extension of Precision-Recall curve to multi-class")
plt.show()
#ROC y AUC
fpr= dict()
tpr= dict()
roc_auc= dict()
for i in range(n_classes):
    fpr[i], tpr[i], _= metrics.roc_curve(y[:,i], y_score[1,i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
# Graficamos ROC
# Plot of a ROC curve for a specific class
for i,rotura in zip(range(n_classes),roturas):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(rotura)
    plt.legend(loc="lower right")
    plt.show()

% de aciertos sobre el set de evaluación: 1.0
% de aciertos sobre el set de evaluación: 1.0


TypeError: list indices must be integers or slices, not tuple

In [ ]:
y_score

[array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]])]